In [1]:
# Your solution here
# import and spark session creation cell
import sys
sys.path.append('/home/aman/programs/gitrepos/PySpark/flight_data_analysis')
from datalist import flights_dataset
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, LongType

spark = SparkSession.builder.appName("DataFrame Example").getOrCreate()

schema = StructType([StructField("Origin_airport",StringType(), True),
                     StructField("Destination_airport", StringType(), True),
                     StructField("Origin_city", StringType(), True),
                     StructField("Destination_city", StringType(), True),
                     StructField("Passengers", IntegerType(), True),
                     StructField("Seats", IntegerType(), True),
                     StructField("Flights", IntegerType(), True),
                     StructField("Distance", LongType(), True),
                     StructField("Fly_date", DateType(), True),
                     StructField("Origin_population", LongType(), True),
                     StructField("Destination_population", LongType(), True),
                     StructField("Org_airport_lat", StringType(), True),
                     StructField("Org_airport_long", StringType(), True),
                     StructField("Dest_airport_lat", StringType(), True),
                     StructField("Dest_airport_long", StringType(), True),
                     
                     ])

print(flights_dataset)
dataframe = spark.read.options(header=True, nullValue='NA').csv(flights_dataset, schema=schema)

print("Total row count: ", dataframe.count())
dataframe.printSchema()
dataframe.show(5)

25/05/17 15:54:48 WARN Utils: Your hostname, Nothing-Is-Real resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/17 15:54:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/17 15:54:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


/home/aman/datasets/Airports2.csv


Total row count:  3606803
root
 |-- Origin_airport: string (nullable = true)
 |-- Destination_airport: string (nullable = true)
 |-- Origin_city: string (nullable = true)
 |-- Destination_city: string (nullable = true)
 |-- Passengers: integer (nullable = true)
 |-- Seats: integer (nullable = true)
 |-- Flights: integer (nullable = true)
 |-- Distance: long (nullable = true)
 |-- Fly_date: date (nullable = true)
 |-- Origin_population: long (nullable = true)
 |-- Destination_population: long (nullable = true)
 |-- Org_airport_lat: string (nullable = true)
 |-- Org_airport_long: string (nullable = true)
 |-- Dest_airport_lat: string (nullable = true)
 |-- Dest_airport_long: string (nullable = true)

+--------------+-------------------+-------------+----------------+----------+-----+-------+--------+----------+-----------------+----------------------+----------------+-----------------+----------------+-----------------+
|Origin_airport|Destination_airport|  Origin_city|Destination_city|P

# Flight Data Analysis - Advanced Questions Set 1
This notebook contains 25 advanced questions designed to deepen your understanding of PySpark DataFrame operations and functions.

## Question 1
Find the top 10 airports with the highest average number of passengers per flight.

In [10]:
# Your solution here
import pyspark.sql.functions as f


df = dataframe.groupBy('Origin_airport').agg(f.sum('Passengers').alias('Passengers'))
df1 = dataframe.groupBy('Destination_airport').agg(f.sum('Flights').alias('Flights'))

#Taking care of flights which might only be terminals
df2 = df.join(df1, df.Origin_airport == df1.Destination_airport, 'fullouter').groupBy('Origin_airport').agg(f.avg(f.col('Passengers') / f.col('Flights')).alias('Average_Passengers')).orderBy('Average_Passengers', ascending=False)

df2.show(5)

[Stage 32:====>                                                   (1 + 11) / 12]

+--------------+------------------+
|Origin_airport|Average_Passengers|
+--------------+------------------+
|           NQX| 378.1904761904762|
|           SHN|             297.0|
|           RCA|280.40425531914894|
|           AZA|132.12900457665904|
|           HNL|129.30888068285006|
+--------------+------------------+
only showing top 5 rows



## Question 2
Calculate the total number of flights and passengers for each day of the week.

In [11]:
# Your solution here
df = dataframe.select(f.year('Fly_date').alias('Year'), f.weekofyear('Fly_date').alias('WeekNo'), 'Passengers', 'Flights').groupBy('Year', 'WeekNo').agg(f.sum('Passengers'), f.sum('Flights')).orderBy(['Year', 'WeekNo'], ascending=[0,0])
df.show(10)


[Stage 40:============================>                            (6 + 6) / 12]

+----+------+---------------+------------+
|Year|WeekNo|sum(Passengers)|sum(Flights)|
+----+------+---------------+------------+
|2009|    49|       42374318|      587123|
|2009|    44|       41686490|      576901|
|2009|    40|       45051011|      602156|
|2009|    36|       41339351|      581047|
|2009|    31|       48286811|      632620|
|2009|    27|       50708453|      644983|
|2009|    23|       47754907|      619372|
|2009|    18|       45229482|      614442|
|2009|    14|       44485987|      601592|
|2009|     9|       45033607|      615601|
+----+------+---------------+------------+
only showing top 10 rows



## Question 3
Identify the routes where the average flight distance is greater than 1000 miles and the average seat occupancy is above 75%.

In [14]:
# Your solution here
df = dataframe.groupBy(['Origin_airport', 'Destination_airport']).agg(f.avg('Distance').alias('Average_distance'), (f.sum('Passengers')/ f.sum('Seats')).alias('Average_occupancy')).filter(f.col('Average_occupancy') > 0.75)
df.show(10)

# just an observation out of curiosity
print(df.corr('Average_distance', 'Average_occupancy'))
"""
It seems there is an inverse relation between the above two values with a correlation value of -0.0022715273511054505
"""

+--------------+-------------------+----------------+------------------+
|Origin_airport|Destination_airport|Average_distance| Average_occupancy|
+--------------+-------------------+----------------+------------------+
|           MOD|                EKO|           354.0|0.8556443556443556|
|           RAP|                RNO|           917.0|0.9199255121042831|
|           BOS|                RNO|          2524.0|0.8174157303370787|
|           JAN|                RNO|          1729.0|0.8545454545454545|
|           BMI|                FAR|           594.0|0.8668341708542714|
|           AUS|                MCN|           845.0|               1.0|
|           YIP|                TUL|           782.0|              0.95|
|           PNS|                TYR|           502.0|0.8481012658227848|
|           BFI|                CVO|           215.0|0.8888888888888888|
|           TPA|                BGR|          1385.0|0.8247011952191236|
+--------------+-------------------+---------------

[Stage 49:==============================================>         (10 + 2) / 12]

-0.0022715273511054505


## Question 4
Find the month with the highest total number of flights for each year.

In [34]:
# Your solution here

df = dataframe.select(f.year('Fly_date').alias('Year'), f.month('Fly_date').alias('Month'), 'Flights').groupBy('Year', 'Month').agg(f.sum('Flights').alias('Total_flights'))
df = df.groupBy('Year').agg(f.max('Total_flights')).orderBy('Year', ascending=0)
df.show()

[Stage 99:===================================================>    (11 + 1) / 12]

+----+------------------+
|Year|max(Total_flights)|
+----+------------------+
|2009|            644983|
|2008|            694851|
|2007|            717852|
|2006|            715580|
|2005|            746133|
|2004|            741458|
|2003|            703527|
|2002|            697038|
|2001|            618273|
|2000|            603009|
|1999|            574662|
|1998|            551701|
|1997|            534342|
|1996|            534918|
|1995|            541336|
|1994|            499857|
|1993|            474755|
|1992|            462575|
|1991|            445676|
|1990|            446416|
+----+------------------+



In [33]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df = dataframe.groupBy(F.year('Fly_date').alias('Year'), F.month('Fly_date').alias('Month')).agg(F.sum('Flights').alias('Total_Flights'))

window_spec = Window.partitionBy('Year')
df_with_max = df.withColumn('Max_Flights', F.max('Total_Flights').over(window_spec))
result = df_with_max.filter(F.col('Total_Flights') == F.col('Max_Flights')).select('Year', 'Month', 'Total_Flights').orderBy('Year', ascending=0)
result.show()

[Stage 93:===================================================>    (11 + 1) / 12]

+----+-----+-------------+
|Year|Month|Total_Flights|
+----+-----+-------------+
|2009|    7|       644983|
|2008|    7|       694851|
|2007|    8|       717852|
|2006|    8|       715580|
|2005|    8|       746133|
|2004|    8|       741458|
|2003|   10|       703527|
|2002|   10|       697038|
|2001|    8|       618273|
|2000|   10|       603009|
|1999|    8|       574662|
|1998|    7|       551701|
|1997|    8|       534342|
|1996|    8|       534918|
|1995|    8|       541336|
|1994|    8|       499857|
|1993|    8|       474755|
|1992|    8|       462575|
|1991|   10|       445676|
|1990|    8|       446416|
+----+-----+-------------+



## Question 5
Calculate the average delay time per airline (assuming a delay column exists). If delay data is not available, suggest how you would incorporate it.

In [29]:

#Your solution here or explanation
"""
delay column doesn't exists in the current data.
To calulate the delay, we need departure time and actual departure time, also arrival time and actual arrival time
Since none of the columns are present in the dataset we need to skip the solution for the query as it can not be calculated for the given dataset.

Loading different dataset which have arrival and departure timings.

Columns to be used are: ArrDelay (Arrival delay), DepDelay (Departure delay) wrt FlightNum which will be the identifier instead of airline which is not available.
We could potentially create column airline wrt FlightNum by identifying them with their respective airline which need to be provided externally. So using FlightNum directly.

"""
import datalist
from importlib import reload

reload(datalist)
from datalist import flight_data_2008

delay_data = spark.read.options(header=True, inferschema=True, nullValue='NA').csv(flight_data_2008)

delay_data.printSchema()

#filtering data to exlude flights with no delays and early arrival/departure
df = delay_data.select('FlightNum', 'ArrDelay', 'DepDelay').where((f.col('ArrDelay') > 0) & (f.col('DepDelay') > 0))

df.groupBy('FlightNum').agg(f.avg(f.col('ArrDelay') + f.col('DepDelay'))).show()

#although the above solution give result, it might be incorrect as both arrival delay an dep delay can be 
# completely independent and might not be accurate when tied together to create the average.

root
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- DepTime: integer (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- ArrTime: integer (nullable = true)
 |-- CRSArrTime: integer (nullable = true)
 |-- UniqueCarrier: string (nullable = true)
 |-- FlightNum: integer (nullable = true)
 |-- TailNum: string (nullable = true)
 |-- ActualElapsedTime: integer (nullable = true)
 |-- CRSElapsedTime: integer (nullable = true)
 |-- AirTime: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Distance: integer (nullable = true)
 |-- TaxiIn: integer (nullable = true)
 |-- TaxiOut: integer (nullable = true)
 |-- Cancelled: integer (nullable = true)
 |-- CancellationCode: string (nullable = true)
 |-- Diverted: integer (nullable = true)

[Stage 84:============================>                            (6 + 6) / 12]

+---------+--------------------------+
|FlightNum|avg((ArrDelay + DepDelay))|
+---------+--------------------------+
|     1580|         60.19130434782609|
|      463|          91.7840616966581|
|     2366|         80.33806818181819|
|      833|         79.92128279883381|
|      148|          81.3467843631778|
|     2866|        110.42817679558011|
|     1342|          71.8340425531915|
|      496|                   63.3584|
|     3918|         66.28372093023256|
|     1829|         92.67772511848341|
|     1088|         82.71245421245422|
|     1591|          83.6988416988417|
|     2122|                62.7421875|
|     1238|         93.05466970387243|
|     3749|         69.45614035087719|
|     7754|         67.60975609756098|
|     2659|         68.66844919786097|
|     2142|          98.6842105263158|
|     7240|        105.85964912280701|
|     7253|         134.4909090909091|
+---------+--------------------------+
only showing top 20 rows



## Question 6
Determine the top 5 origin cities with the most diverse set of destination cities.

In [ ]:
# Your solution here

## Question 7
Find the correlation between the number of flights and the population of the origin city.

In [ ]:
# Your solution here

## Question 8
Identify the routes with the highest variance in passenger numbers across months.

In [ ]:
# Your solution here

## Question 9
Calculate the average number of flights per day for each airport.

In [ ]:
# Your solution here

## Question 10
Find the routes where the average number of seats per flight has increased year over year.

In [ ]:
# Your solution here

## Question 11
Determine the busiest hour of the day for flights (assuming a time column exists). If not, suggest how to analyze this.

In [ ]:
# Your solution here or explanation

## Question 12
Calculate the average flight distance for routes with more than 100 flights.

In [ ]:
# Your solution here

## Question 13
Find the top 5 destination cities with the highest average passenger count per flight.

In [ ]:
# Your solution here

## Question 14
Identify the routes with the longest average flight distance.

In [ ]:
# Your solution here

## Question 15
Calculate the total number of passengers for flights originating from cities with a population greater than 1 million.

In [ ]:
# Your solution here

## Question 16
Find the average number of flights per route for routes with more than 50 flights.

In [ ]:
# Your solution here

## Question 17
Determine the routes with the highest average seat occupancy.

In [ ]:
# Your solution here

## Question 18
Calculate the average delay per route (assuming delay data is available). If not, describe how you would approach this.

In [ ]:
# Your solution here or explanation

## Question 19
Find the top 10 routes with the highest number of flights during holiday seasons (assuming holiday dates are known).

In [ ]:
# Your solution here or explanation

## Question 20
Identify the routes with the most consistent passenger numbers year over year.

In [ ]:
# Your solution here

## Question 21
Calculate the average number of passengers per flight for each airline (assuming airline data is available).

In [ ]:
# Your solution here or explanation

## Question 22
Find the routes with the highest increase in passenger numbers over the last 5 years.

In [ ]:
# Your solution here

## Question 23
Determine the average number of seats per flight for each month.

In [ ]:
# Your solution here

## Question 24
Identify the top 5 origin airports with the highest average flight distance.

In [ ]:
# Your solution here

## Question 25
Calculate the total number of flights canceled (assuming cancellation data is available). If not, describe how you would track cancellations.

In [ ]:
# Your solution here or explanation